# Wed. 29th or Tues 28 of Nov

Mike and I met in my office. Looked at annual plots of inventories.
Then saw a big dip in annual data that will be missed on cencus data.
So, here we model annual data.


On Dec 5. Mike and I had a follow up and talked about the next step.

- **Hypothesis** Decline in inventory from time $t$ to $t+1$ if ```NPP``` at $t$ was below average.

What about lag tho? He had mentioned earlier maybe people make changes 3 years after a drought.

- Annual State Level
- Add Washington, Utah, Arizona, Nevada
- Find some examples that inventory goes down sharply at time $t+1$ and look at NPP at time $t$.

- $y-$variable should be deltas: $y_{t+1} = I_{t+1} - I_t$ where $I$ is for inventory.
Under this scenario independent variables can be also deltas or $x_t$ corresponds to $y_{t+1}$. In this notebook
we will go with the latter scenario.


In [8]:
import shutup
shutup.please()

import pandas as pd
import numpy as np
from datetime import datetime
import os, os.path, pickle, sys
import seaborn as sns

import matplotlib
import matplotlib.pyplot as plt

sys.path.append("/Users/hn/Documents/00_GitHub/Rangeland/Python_Codes/")
import rangeland_core as rc

In [9]:
data_dir_base = "/Users/hn/Documents/01_research_data/RangeLand/Data/"
census_population_dir = data_dir_base + "census/"
# Shannon_data_dir = data_dir_base + "Shannon_Data/"
# USDA_data_dir = data_dir_base + "/NASS_downloads/"
param_dir = data_dir_base + "parameters/"
Min_data_base = data_dir_base + "Min_Data/"
reOrganized_dir = data_dir_base + "reOrganized/"

# Read

In [10]:
SoI = ["Alabama", "Arizona", "Arkansas", "California", 
       "Colorado", "Florida", "Georgia", 
       "Idaho", "Illinois", "Iowa", 
       "Kansas", "Kentucky", "Louisiana", 
       "Mississippi", "Missouri", "Montana", 
       "Nebraska", "Nevada", "New Mexico", "North Dakota", 
       "Oklahoma", "Oregon", "South Dakota", 
       "Tennessee", "Texas", "Utah", "Virginia", "Washington",
       "Wyoming"]

abb_dict = pd.read_pickle(param_dir + "state_abbreviations.sav")
SoI_abb = []
for x in SoI:
    SoI_abb = SoI_abb + [abb_dict["full_2_abb"][x]]

In [11]:
county_id_name_fips = pd.read_csv(Min_data_base + "county_id_name_fips.csv")
county_id_name_fips.rename(columns=lambda x: x.lower().replace(' ', '_'), inplace=True)

county_id_name_fips = county_id_name_fips[county_id_name_fips.state.isin(SoI_abb)].copy()

county_id_name_fips.sort_values(by=["state", "county"], inplace=True)

county_id_name_fips = rc.correct_Mins_FIPS(df=county_id_name_fips, col_="county")
county_id_name_fips.rename(columns={"county": "county_fips"}, inplace=True)

county_id_name_fips.reset_index(drop=True, inplace=True)
print (len(county_id_name_fips.state.unique()))
county_id_name_fips.head(2)

29


,county_fips,county_name,fips,state
0,01001,Autauga County,1001,AL
1,01003,Baldwin County,1003,AL


In [12]:
county_id_name_fips["state_fip"] = county_id_name_fips.county_fips.str.slice(0, 2)
county_id_name_fips.head(2)

,county_fips,county_name,fips,state,state_fip
0,01001,Autauga County,1001,AL,01
1,01003,Baldwin County,1003,AL,01


In [15]:
herbRatio = pd.read_csv(data_dir_base + "Supriya/Nov30_HerbRatio/state_herb_ratio.csv")
herbRatio = rc.correct_state_int_fips_to_str(df=herbRatio, col_="state_fip")
herbRatio.sort_values(by=["state_fip"], inplace=True)
herbRatio.dropna(how="any", inplace=True)
herbRatio.head(2)

,state_fip,ag_land,AWATER,pixel_count,herb_avg,herb_std
8,01,131174048583,4593327154,10874.0,21.019864,25.356867
28,04,294198551143,1027337603,3704408.0,45.168734,17.311673


In [16]:
NPP = pd.read_csv(Min_data_base + "statefips_annual_MODIS_NPP.csv")
NPP.rename(columns={"NPP": "modis_npp",
                    "statefips90m": "state_fip"}, 
           inplace=True)
NPP = rc.correct_3digitStateFips_Min(NPP, "state_fip")

NPP = NPP[NPP.state_fip.isin(county_id_name_fips.state_fip)]
NPP.reset_index(drop=True, inplace=True)

NPP.head(2)

,year,state_fip,modis_npp
0,2001,01,0.816597
1,2001,04,0.115683


In [17]:
# Rangeland area and Total area:
state_RA = pd.read_pickle(reOrganized_dir + "state_RA_area.sav")
state_RA = state_RA["state_RA_area"]
print (len(state_RA.state_fip.unique()) == len(state_RA.state_fip))
state_RA.head(2)

True


,state_fip,rangeland_acre,state_area_acre
0,01,31570.99,14825637.76
1,04,54476918.89,72998287.36


In [18]:
print (len(NPP.state_fip.unique()))
print (len(state_RA.state_fip.unique()))

29
49


In [19]:
NPP.head(2)

,year,state_fip,modis_npp
0,2001,01,0.816597
1,2001,04,0.115683


In [20]:
state_RA.head(2)

,state_fip,rangeland_acre,state_area_acre
0,01,31570.99,14825637.76
1,04,54476918.89,72998287.36


In [21]:
state_annual_NPP_Ra = pd.merge(NPP, state_RA, on = ["state_fip"], how = "left")
state_annual_NPP_Ra.head(2)

,year,state_fip,modis_npp,rangeland_acre,state_area_acre
0,2001,01,0.816597,31570.99,14825637.76
1,2001,04,0.115683,54476918.89,72998287.36


In [22]:
shannon_annual = pd.read_csv(reOrganized_dir + "Beef_Cows_fromCATINV.csv")
shannon_annual = shannon_annual[shannon_annual.state.isin(SoI_abb)]
shannon_annual.head(2)

,state,1920,1921,1922,1923,1924,1925,1926,1927,1928,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,AL,103000.0,106000.0,108000.0,111000.0,114000.0,104000.0,100000.0,96000.0,94000.0,...,680000.0,701000.0,681000.0,652000.0,673000.0,693000.0,714000.0,705000.0,696000.0,697000.0
2,AZ,819000.0,800000.0,795000.0,790000.0,783000.0,714000.0,608000.0,506000.0,445000.0,...,190000.0,200000.0,192000.0,195000.0,195000.0,195000.0,200000.0,215000.0,194000.0,194000.0


In [23]:
wanted_years = np.arange(state_annual_NPP_Ra.year.min(), state_annual_NPP_Ra.year.max())
print (wanted_years[:3])
print (wanted_years[-3:])
cols_ = ["state"] + list(wanted_years.astype(str))
shannon_annual = shannon_annual[cols_]


shannon_annual = shannon_annual[shannon_annual.state.isin(list(county_id_name_fips.state.unique()))]
shannon_annual.sort_values(by=["state"], inplace=True)
shannon_annual.reset_index(drop=True, inplace=True)
shannon_annual.head(2)

[2001 2002 2003]
[2017 2018 2019]


,state,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,AL,737000.0,750000.0,761000.0,732000.0,724000.0,686000.0,687000.0,667000.0,668000.0,669000.0,659000.0,680000.0,701000.0,681000.0,652000.0,673000.0,693000.0,714000.0,705000.0
1,AR,923000.0,927000.0,949000.0,982000.0,967000.0,899000.0,921000.0,943000.0,906000.0,937000.0,928000.0,919000.0,851000.0,862000.0,863000.0,893000.0,914000.0,924000.0,935000.0
